In [22]:
# Settings

# General settings
data_folder = "/home/andrea/CIMA/PROJECT_Volta/data/integrated_data/" #"/home/andrea/Desktop/Test/series/" 
start_analysis = "2018-01-01 00:00"
end_analysis = "2020-12-31 23:00"
freq="D"

# Hmc settings
domain = "volta"
#hmc_output = "/home/andrea/CIMA/PROJECT_Volta/data/rainfall_analysis/imerg/hmc.hydrograph.txt"
#hmc_static_gridded = "/home/andrea/CIMA/PROJECT_Volta/data/rainfall_analysis/data_static/gridded/"
#hmc_static_point = "/home/andrea/CIMA/PROJECT_Volta/data/rainfall_analysis/data_static/point/"

hmc_output = "/home/andrea/CIMA/PROJECT_Volta/data/rainfall_analysis/red_volta/hmc.hydrograph.txt"
hmc_static_gridded = "/home/andrea/CIMA/PROJECT_Volta/data/rainfall_analysis/data_static/gridded/"
hmc_static_point = "/home/andrea/CIMA/PROJECT_Volta/data/rainfall_analysis/red_volta/"

# Data
sm_series = "/home/andrea/CIMA/PROJECT_Volta/data/rainfall_analysis/red_volta/average_SM.txt"
rainfall_series = "/home/andrea/CIMA/PROJECT_Volta/data/rainfall_analysis/red_volta/average_rainfall_mm_h.txt"

In [23]:
# Imports
%matplotlib inline

from ipywidgets import interactive,HBox
import pandas as pd
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt
from pysheds.grid import Grid

import ipywidgets as widgets
from IPython.display import display, HTML
import os, glob
import datetime as dt
import matplotlib.lines as mlines

# Define custom functions
def create_df(choices):
    df = pd.DataFrame(index=pd.date_range(start_time,end_time,freq=freq), columns=choices)
    for name in choices:
        series = pd.read_csv(os.path.join(data_folder, name + ".csv"), index_col=0, header=0, parse_dates=True)
        df[name] = series.reindex(pd.date_range(start_time,end_time,freq=freq), method=None)
    return df

def multiplot(widg):
    choices = widg['new']
    df = create_df(choices)
    data = df.loc[:, choices] if choices else df
    output.clear_output(wait=True)
    with output:
        ax = data.plot(figsize=(10,7))
        plt.show()
        
def combinedplot(widg):
    choices = widg['new']
    data_sm = series["SM"].loc[:, choices] if choices else series
    data_rain = series["rain"].loc[:, choices] if choices else series
    data_qMod = series["discharge_mod"].loc[:, choices] if choices else series
    data_qObs = series["discharge_obs"].loc[:, choices] if choices else series
    output2.clear_output(wait=True)
    with output2:
        ax = data_sm.plot(figsize=(15,7), color='c') 
        plt.ylim(bottom=0)
        plt.ylabel("%")
        axx = ax.twinx()
        ax1 = data_rain.plot(figsize=(15,7), color='g')
        ax2 = data_qMod.plot(figsize=(15,7), color='b')
        ax3 = data_qObs.plot(figsize=(15,7), color='r', style='.')
        plt.ylim(bottom=0)
        #plt.xlim([dt.datetime(2018,1,1),dt.datetime(2019,1,1)])
        plt.ylabel("mm")
        cyan_line = mlines.Line2D([], [], color='cyan', label='sm')
        green_line = mlines.Line2D([], [], color='green', label='rain')
        blue_line = mlines.Line2D([], [], color='blue', label='mod_dis')
        red_line = mlines.Line2D([], [], color='red', label='obs_dis')
        
        plt.legend(handles=[cyan_line, green_line, blue_line, red_line])
        plt.show()

def read_discharge_hmc(file='', col_names=None):
    custom_date_parser = lambda x: dt.datetime.strptime(x, "%Y%m%d%H%M")
    hmc_discharge_df = pd.read_csv(file, header=None, delimiter=r"\s+", parse_dates=[0], index_col=[0], date_parser=custom_date_parser)
    hmc_discharge_df.columns = col_names
    return hmc_discharge_df

In [24]:
# Read static inputs

# Sections
sections = tabular = pd.read_csv(os.path.join(hmc_static_point, domain + ".info_section.txt"), sep="\s+", header=None)
rHMC, cHMC, basin_name, section_name = tabular.values[:,0], tabular.values[:,1], tabular.values[:,2], tabular.values[:,3]

# Pointers
grid = Grid.from_ascii(os.path.join(hmc_static_gridded, domain + ".pnt.txt"))
pnt = grid.read_ascii(os.path.join(hmc_static_gridded, domain + ".pnt.txt"), dtype=np.int8)
areacell = grid.read_ascii(os.path.join(hmc_static_gridded, domain + ".areacell.txt"))

In [25]:
# Compute basin cell area
basin_area = pd.DataFrame(index=section_name, columns=["Area (m2)"])
dirmap_HMC = (8, 9, 6, 3, 2, 1, 4, 7)

for ix, iy, basin, name in zip(cHMC, rHMC, basin_name, section_name):
        basin = grid.catchment(fdir=pnt, x=ix-1, y=iy-1, dirmap=dirmap_HMC, xytype='index')
        mask = np.where(basin>0, 1, np.nan)
        basin_area.loc[name, "Area (m2)"] = np.nansum(mask*areacell).astype("float32")

In [26]:
# Read hmc output
mod_out = read_discharge_hmc(hmc_output, section_name)

In [27]:
# Set timing
start_time = dt.datetime.strptime(start_analysis, "%Y-%m-%d %H:%M")
end_time = dt.datetime.strptime(end_analysis, "%Y-%m-%d %H:%M")

In [28]:
# Read datasets
series = {}

series["SM"] = pd.read_csv(sm_series, index_col=0, header=0, parse_dates=True)[start_time:end_time]
series["rain"] = pd.read_csv(rainfall_series, index_col=0, header=0, parse_dates=True)[start_time:end_time]
dis_out = mod_out[start_time:end_time]

missing_data = [i for i in section_name if not os.path.isfile(os.path.join(data_folder, i + ".csv"))]
display("WARNING! Data for sections " + ", ".join(missing_data) + " are missing!")
dis_in = create_df([i for i in section_name if i not in missing_data])

'WARNING! Data for sections  are missing!'

In [29]:
# Convert all series to mm/day
series["rain"] = series["rain"].resample("D").sum()
series["discharge_mod"] = pd.DataFrame(index = series["rain"].index, columns = dis_out.columns)
series["discharge_obs"] = pd.DataFrame(index = series["rain"].index, columns = dis_in.columns)
series["availability"] = pd.DataFrame(index = series["rain"].index, columns = dis_in.columns)
series["SM"] = series["SM"] * 100

for name in dis_out.columns:
    series["discharge_mod"][name] = (dis_out[name] * (1000*3600)/float(basin_area.loc[name, "Area (m2)"])).resample("D").sum()

for name in series["discharge_obs"].columns:
    series["discharge_obs"][name] = (dis_in[name] * (1000*3600*24)/float(basin_area.loc[name, "Area (m2)"]))
    series["availability"][name] = dis_in[name].resample("D").count()

In [30]:
series_annual = {}

for stype in series.keys():
    series_annual[stype] = series[stype].resample("Y").sum()
    index = pd.Index([str(y) + "_" + stype for y in np.unique(series_annual[stype].index.year)])
    series_annual[stype].set_index(index, inplace=True)

In [31]:
#out_table = pd.concat([series_annual[i].T for i in series_annual.keys()], axis=1)
#out_table = out_table.reindex(sorted(out_table.columns), axis=1)

#display(out_table)
#out_table.to_csv("/home/andrea/Desktop/summary_balance_old.csv")

In [32]:
# Generate list
selector2 = widgets.Dropdown(
options=series_annual["SM"].T.index,
value=series_annual["SM"].T.index[0])

output2 = widgets.Output()

# Set layout and display
form_item_layout = widgets.Layout(
    display='flex',
    justify_content='space-between'
)
display(widgets.VBox([selector2, output2], layout=form_item_layout))

# Re-generate multiplot
selector2.observe(combinedplot, names='value')